In [1]:
import pandas as pd
import numpy as np

#statistics packages
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel
from scipy.stats import spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols

#plot packages
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pip  install openpyxl


The following command must be run outside of the IPython shell:

    $ pip install openpyxl

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


<font face="Arial" size=6 color=#0099ff> 1载入并组合Edata和基础的人口学数据（年龄age,性别gender...)
</font> 
<font face="Arial" size=5 color=#DC143C> tips: </font>
1. 使用 read_csv() 和 to_csv()载入并保存数据 
2. 使用 df1.set_index(’Subject’).join(df2.set_index(’Subject’)) 通过索引合并表格, 也可以使用 'on=column name' 
3. 将 'Subject ID' 设置为索引是方便的 
4. 在对数据操作完毕之后(e.g. join, merge, groupby), 使用df.describe() 和 head()检查数据

 **载入行为数据(Edata的excel版本)**

In [3]:
# 读取Edata的excel版本, pandas.read_excel

se_all = pd.read_excel('/home/mw/input/python5286/SE_all_demo1118.xlsx')
# se_all = pd.read_excel('SE_all_demo1118.xlsx')
raw_data = se_all

In [4]:
raw_data.head()#查看前五行数据

,ExperimentName,Subject,Session,Clock.Information,DataFile.Basename,Display.RefreshRate,ExperimentVersion,Group,RandomSeed,RuntimeCapabilities,...,word.ACC,word.CRESP,word.DurationError,word.OnsetDelay,word.OnsetTime,word.OnsetToOnsetTime,word.RESP,word.RT,word.RTTime,word_no
0,SE_V4_run1,1,1,"<?xml version=""1.0""?>\n<Clock xmlns:dt=""urn:sc...",SE_V4_run1-1-1,60.006,1.0.0.40,1,433954761,Standard,...,1,NaN,0,100,32964,1016,NaN,0,0,37_w_10 Mixdown 1.wav
1,SE_V4_run1,1,1,"<?xml version=""1.0""?>\n<Clock xmlns:dt=""urn:sc...",SE_V4_run1-1-1,60.006,1.0.0.40,1,433954761,Standard,...,1,NaN,0,100,44279,1017,NaN,0,0,54_w_10 Mixdown 1.wav
2,SE_V4_run1,1,1,"<?xml version=""1.0""?>\n<Clock xmlns:dt=""urn:sc...",SE_V4_run1-1-1,60.006,1.0.0.40,1,433954761,Standard,...,1,NaN,0,100,57595,1016,NaN,0,0,07_w_10 Mixdown 1.wav
3,SE_V4_run1,1,1,"<?xml version=""1.0""?>\n<Clock xmlns:dt=""urn:sc...",SE_V4_run1-1-1,60.006,1.0.0.40,1,433954761,Standard,...,1,NaN,0,99,70893,1017,NaN,0,0,41_w_10 Mixdown 1.wav
4,SE_V4_run1,1,1,"<?xml version=""1.0""?>\n<Clock xmlns:dt=""urn:sc...",SE_V4_run1-1-1,60.006,1.0.0.40,1,433954761,Standard,...,1,NaN,0,100,84326,1016,NaN,0,0,14_w_10 Mixdown 1.wav


In [5]:
#atb_list = ['ExperimentName','Subject','Block','answer.ACC','answer.CRESP',  
#            'Trigger.OnsetTime','answer.OnsetTime','answer.RESP','answer.RT',  
#            'answer.RTTime','answer1','answer2','answer3','answer4','condition',
#            'FA','repot.RESP','repot.RT','repot.RTTime','rightA',  
#            'run','sen','sen_no','word']

In [6]:
#set_index, rename, join
data = raw_data.set_index(['Subject'])
# replace wrong named data including {63:20,99:11,152:15,217:17}
replaced_pairs = {63:20,99:11,152:15,217:17}
data = data.rename(replaced_pairs, axis='index')
#se_basic_infor = pd.read_excel('SE_basic_infor_demo1118.xlsx')
se_basic_infor = pd.read_excel('/home/mw/input/python5286/SE_basic_infor_demo1118.xlsx')
se_basic_infor = se_basic_infor.set_index('Subject')
data = data.join(se_basic_infor) #
data.describe()

,Session,Display.RefreshRate,Group,RandomSeed,Trigger.ACC,Trigger.CRESP,Trigger.DurationError,Trigger.OnsetDelay,Trigger.OnsetTime,Trigger.OnsetToOnsetTime,...,word.DurationError,word.OnsetDelay,word.OnsetTime,word.OnsetToOnsetTime,word.RESP,word.RT,word.RTTime,LSHS_total,LSHS_AVH,age
count,450.0,450.000000,450.0,4.500000e+02,450.0,0.0,450.0,450.000000,450.00000,450.0,...,450.0,450.000000,450.000000,450.000000,0.0,450.0,450.0,360.000000,360.000000,360.000000
mean,1.0,60.005267,1.0,3.995249e+08,0.0,NaN,-999999.0,28.133333,16438.80000,0.0,...,0.0,89.573333,228847.515556,1016.544444,NaN,0.0,0.0,25.333333,13.666667,24.666667
std,0.0,0.000574,0.0,1.223310e+09,0.0,NaN,0.0,5.076695,13187.44787,0.0,...,0.0,15.626207,116331.834459,0.503022,NaN,0.0,0.0,11.914271,7.419015,0.472061
min,1.0,60.004000,1.0,-1.653807e+09,0.0,NaN,-999999.0,19.000000,7638.00000,0.0,...,0.0,66.000000,27346.000000,1015.000000,NaN,0.0,0.0,15.000000,7.000000,24.000000
25%,1.0,60.005000,1.0,-8.118263e+08,0.0,NaN,-999999.0,24.000000,9914.00000,0.0,...,0.0,83.000000,128547.000000,1016.000000,NaN,0.0,0.0,15.000000,7.000000,24.000000
50%,1.0,60.005000,1.0,4.339548e+08,0.0,NaN,-999999.0,31.000000,11666.00000,0.0,...,0.0,83.000000,226937.500000,1017.000000,NaN,0.0,0.0,19.000000,10.000000,25.000000
75%,1.0,60.006000,1.0,1.516501e+09,0.0,NaN,-999999.0,33.000000,18539.00000,0.0,...,0.0,100.000000,328988.250000,1017.000000,NaN,0.0,0.0,42.000000,24.000000,25.000000
max,1.0,60.006000,1.0,2.012595e+09,0.0,NaN,-999999.0,34.000000,62952.00000,0.0,...,0.0,166.000000,463906.000000,1017.000000,NaN,0.0,0.0,42.000000,24.000000,25.000000


<font face="Arial" size=6 color=#0099ff>  2. 检查每个被试的所有试次的反应数据 (反应时 和 选择) </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. 在跨试次平均反应时间之前, 检查试次间反应时间的分布. 检测极端值并尝试处理他们 (subject 52)
2. 检查柱状图. 

In [7]:
index_name=data.index.unique()
for id,_index_name in enumerate(index_name): 
    #将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标，一般用在 for 循环当中
    df = data.loc[data.index==_index_name,'answer.RT']
    plt.subplot(index_name.shape[0],1,id+1)
    sns.histplot(df,bins=20)
    plt.show()

AttributeError: module 'seaborn' has no attribute 'histplot'

<Figure size 432x288 with 1 Axes>

<font face="Arial" size=6 color=#0099ff>  3. 对每一个被试平均反应时和准确率 </font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df[df['column_name']] 来选择列
2. df.mean(level='Sujbect') 来对每个被试跨试次平均反应时. df.fun() (sum, std, sqrt...)来操作数据. 
3. pd.concat((df1,df2),axis=1) 来合并 RT, ACC, False alarm 和 其他变量进一个表. 

In [9]:
#to check RT distribution for each subject and detect outliers
opt =  pd.DataFrame()
bd_pre =  data[data['condition']=='pre']
bd_unp =  data[data['condition']=='unp']
# rt is zero in some trials, exluding them.

opt['acc_pre'] = bd_pre['answer.ACC'].mean(level='Subject')
opt['acc_unp'] = bd_unp['answer.ACC'].mean(level='Subject')

# only for all RIGHT response 
opt['conf_pre'] = bd_pre[bd_pre['answer.ACC']==1]['answer.RESP'].mean(level='Subject')
opt['conf_unp'] = bd_unp[bd_unp['answer.ACC']==1]['answer.RESP'].mean(level='Subject')

opt['rt_pre'] = bd_pre[bd_pre['answer.ACC']==1]['answer.RT'].mean(level='Subject') #
opt['rt_unp'] = bd_unp[bd_unp['answer.ACC']==1]['answer.RT'].mean(level='Subject') #

# add false alarm rate
bd_unp_ans_fa = bd_unp[bd_unp['answer.RESP']==bd_unp['FA']][['answer.RESP']]
bd_unp_ans_fa['count_num'] = 1
opt['fa_unp'] = bd_unp_ans_fa['count_num'].sum(level='Subject')
opt['fa_pre'] = 0

#concat different condtions and different measurements. 

acc = pd.concat((opt['acc_pre'],opt['acc_unp']),axis = 0)#axis=0可以理解为按行合并
conf = pd.concat((opt['conf_pre'],opt['conf_unp']),axis = 0)                  
rt = pd.concat((opt['rt_pre'],opt['rt_unp']),axis = 0) 
fa = pd.concat((opt['fa_pre'],opt['fa_unp']),axis = 0) 
opt_all = pd.concat((acc,rt,fa,conf),axis=1)
opt_all = opt_all.rename({0:'acc',1:'rt',2:'fa',3:'conf'},axis = 'columns')

In [10]:
opt_all['condition'] = ['pre']*4+['unp']*4
opt_all = opt_all.join(se_basic_infor)
opt_all.describe()

**基于量表分数将所有的被试划分为两个组，增加一个组变量. **

In [12]:
opt = opt.join(se_basic_infor)
#print(opt)
LSHS_AVH_median = opt['LSHS_AVH'].median()
group = []
for i in opt.index:
    if opt['LSHS_AVH'][i]<=LSHS_AVH_median:
        group.append('Low')
    else:
        group.append('High')
opt['group'] = group 
opt.dropna(how='any')
#opt[['LSHS_AVH','group']]
opt_all = opt_all.join(opt['group'])

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df.iloc[i,j]来选择数据的子集.

In [13]:
# combine with questionaire data

se_scale = pd.read_excel('SE_scale_demo1118.xlsx')
# se_scale = pd.read_excel('SE_scale.xlsx')
# se_scale = pd.read_excel('/home/mw/input/python5286/SE_scale_demo1118.xlsx')
se_scale  = se_scale.set_index('Subject')
#print(se_scale)
#se_scale = se_scale.set_index(['Subject'])
se_scale = se_scale.iloc[:,7:76]
opt_all = opt_all.join(se_scale,rsuffix='_r')

In [14]:
se_qpe = pd.read_excel('SE-QPE_40_demo1118.xlsx')
# se_qpe = pd.read_excel('/home/mw/input/python5286/SE-QPE_40_demo1118.xlsx')
se_qpe = se_qpe.set_index('Subject')
QPE_severity = pd.DataFrame(se_qpe.iloc[:,12:16].sum(axis = 1),columns = ['QPE_severity'])
opt_all = opt_all.join(QPE_severity)

<font face="Arial" size=6 color=#0099ff>  4. 对每一个被试计算所有项目分数的总和. </font>

In [15]:
# calculate and summary the sales into sum scores.
#print(opt_all.columns)
stai_all = range(1,21)
stai_all_opp = [1,3,6,7,10,13,14,16,19]
stai_all_pos = list(set(stai_all).difference(set(stai_all_opp)))

STAI_oppsiteiterm_colnames = ['STAI'+str(i) for i in stai_all_opp]
STAI_positerm_colnames = ['STAI'+str(i) for i in stai_all_pos]
opt_all['TAI_all'] = 45-opt_all.loc[:,STAI_oppsiteiterm_colnames].sum(axis=1)+opt_all.loc[:,STAI_positerm_colnames].sum(axis=1)
#print(opt_all.loc[:,STAI_positerm_colnames])
#print(x)
BDI_colnames = ['BDI'+str(i+1) for i in range(21)]
HPSV_colnames = ['HPSV'+str(i+1) for i in range(9)]
RAHQ_colnames = ['RAHQ'+str(i+1) for i in range(18)]

opt_all['BDI_all'] = opt_all.loc[:,BDI_colnames].sum(axis=1)
opt_all['HPSV_all'] = opt_all.loc[:,HPSV_colnames].sum(axis=1)
opt_all['RAHQ_all'] = opt_all.loc[:,RAHQ_colnames].sum(axis=1)

**Drop some subjects.**

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. df.drop() 来删除列或行

In [16]:
opt_all

In [17]:
#opt_all = opt_all.drop([53,34])# also remove sub34 because no avh basic infors.
#rt, zero exist. sub42(30 zeros)，44(60 zeros)，52(90 and more zeros)
print(opt_all[opt_all['condition']=='pre']['acc'])
opt_all[np.abs(opt_all.acc-opt_all.acc.mean())<=(3*opt_all.acc.std())] 
opt_all[~(np.abs(opt_all.acc-opt_all.acc.mean())>(3*opt_all.acc.std()))] 
opt_all[opt_all.acc<0.25]

<font face="Arial" size=6 color=#0099ff>  5. 对性别，年龄，调查问卷检查分布和个体的数据点</font>

In [18]:
df = opt_all[['age','group']]
sns.histplot(df,bins=20)

In [19]:
df_bygroup_bd = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']].groupby(['group']).mean()
y_err_bd = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']].groupby(['group']).sem()
df_bygroup_bd = df_bygroup_bd.T
y_err_bd = y_err_bd.T

df_bygroup_bd.plot(kind = 'bar',yerr = y_err_bd,width=0.8,style=['r','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
plt.ylabel('bd')
plt.show()

In [20]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all']]
for id, _scale in enumerate(df.columns):
    df_scale = df.loc[:,str(_scale)]
    plt.subplot(df.columns.shape[0],1,id+1)
    sns.histplot(df_scale,bins=20)
    plt.show()

In [21]:
df_all = df.drop_duplicates()
print(df_all.columns)
scale_name=df_all.columns

In [22]:
d = np.array(df_all[scale_name].dropna())
print(d)
sns.set(style="white", palette="muted", color_codes=True)

# Set up the matplotlib figure
f, axes = plt.subplots(figsize=(7, 7), sharex=True)
sns.despine(left=True)
# Plot a historgram and kernel density estimate
sns.distplot(d, color="m")
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()

**检查量表分数的分布和相关**

In [26]:
df

In [27]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity']]#,'TAI_all','BDI_all'
sns.pairplot(df,dropna=True)

**按照条件和组水平画准确率分布**

In [28]:
df_groupby = opt_all[['acc','group','condition']].groupby(['group','condition'])
print(df_groupby.groups.keys())
df_list = [df_groupby.get_group(key).reset_index()['acc'].rename() for key in df_groupby.groups.keys()]
df = pd.concat(df_list,axis=1)
#df = df.rename(columns={1:})
sns.histplot(df,bins=20)
plt.show()

<font face="Arial" size=6 color=#0099ff>  6. 在群水平(性别，年龄，问卷项目，反应时间，准确率和其他)计算和绘制变量</font>

<font face="Arial" size=5 color=#DC143C> tips: </font>
1. 使用 ** df.groupby() ** 来将数据划分为亚组, 然后是用函数 (平均值, 均值标准误...)来对每个亚组操作数据.

In [30]:
df_bygroup_basicinfor = opt_all[['gender','age','group']].groupby(['group']).mean()
y_err_bd = opt_all[['gender','age','group']].groupby(['group']).sem()
df_bygroup_basicinfor = df_bygroup_basicinfor.T
y_err_bd = y_err_bd.T

df_bygroup_basicinfor.plot(kind = 'bar',yerr = y_err_bd,width=0.8,style=['r','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('bd')
plt.show()

#print(opt_all[['gender','group']])
df_bygroup_basicinfor = opt_all[['age','gender','group']].groupby(['gender','group']).count()/2
print(df_bygroup_basicinfor)
#y_err_bd = opt_all[['gender','group']].groupby(['group']).sem()
df_bygroup_basicinfor = df_bygroup_basicinfor.T
#y_err_bd = y_err_bd.T

df_bygroup_basicinfor.plot(kind = 'bar',width=0.8,style=['r','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('bd')
plt.show()

In [31]:
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']]
df = df.drop_duplicates()
# color_list = ['rgba(93, 164, 214, 0.5)', 'rgba(93, 164, 214, 1)',  
#               'rgba(44, 160, 101, 0.5)', 'rgba(44, 160, 101, 1)',  
#               'rgba(255, 65, 54, 0.5)', 'rgba(255, 65, 54, 1)',  
#               'rgba(207, 114, 255, 0.5)','rgba(207, 114, 255, 1)',  
#               'rgba(127, 96, 0, 0.5)','rgba(127, 96, 0, 1)',  
#               'rgba(255, 165, 0,0.5)','rgba(255, 165, 0,1)',  
#               'rgba(255, 20, 147,0.5)','rgba(255, 20, 147,1)']
color_list = ['rgba(93, 164, 214, 5)', 'rgba(93, 214, 214, 1)',  
              'rgba(44, 160, 101, 5)', 'rgba(44, 210, 101, 1)',  
              'rgba(255, 65, 54, 5)', 'rgba(255, 115, 54, 1)',  
              'rgba(207, 114, 255, 5)','rgba(207, 164, 255, 1)',  
              'rgba(127, 96, 0, 5)','rgba(127, 146, 0, 1)',  
              'rgba(255, 165, 0,5)','rgba(255, 215, 0,1)',  
              'rgba(255, 20, 147,5)','rgba(255, 70, 147,1)']
scale_list = ['LSHS_total','LSHS_total','LSHS_AVH','LSHS_AVH','HPSV_all','HPSV_all','RAHQ_all','RAHQ_all',  
              'QPE_severity','QPE_severity','TAI_all','TAI_all','BDI_all','BDI_all']
group_list = ['High','Low']*7
df_gls = pd.DataFrame({'gls':group_list,
                       'sls':scale_list,
                       'cls':color_list})
data = pd.DataFrame([
            {
                'y': df[df['group']==group][clms],
                'type': 'box', 
                'boxpoints':'all',
                #'fillcolor':
                'marker': dict(color = df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0]),
                'boxmean':'sd',
                'name': clms + '_' + group
            } 
                for clms in ['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all']
            for group in ['High','Low']]
)
#iplot(Figure(data=Data(data),layout = Layout(showlegend=False)))

**检查组间量表分数的差异**

In [32]:
# get a list of all columns in the dataframe without the Group column
df = opt_all[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all','QPE_severity','TAI_all','BDI_all','group']]
df = df.drop_duplicates()
column_list = [x for x in df.columns if x != 'group']#列表解析式，返回是一个列表
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    group1 = df.where(df.group== 'High')[column].dropna()
    group2 = df.where(df.group== 'Low')[column].dropna()
    print (group1.count(),group2.count())
    # add the output to the dictionary 
    t_test_results[column] = ttest_ind(group1,group2)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']
results_df

<font face="Arial" size=6 color=#0099ff>  7.画条形图并对反应时和准确率进行方差分析和t检验. </font>

**按照条件和组对准确率画图**

In [33]:
# remove subjects with very high fa, let's say more than 40
opt_all = opt_all[~opt_all.index.isin(opt_all[opt_all['fa']>40].index)]
df_bygroup_acc = opt_all[['acc','group','condition']].groupby(['group','condition']).mean()
y_err_acc = opt_all[['acc','group','condition']].groupby(['group','condition']).sem()
df_bygroup_acc = df_bygroup_acc.T
y_err_acc = y_err_acc.T

df_bygroup_acc.plot(kind = 'bar',yerr = y_err_acc,width=0.8,style=['r--','r','b--','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
#ylabel('acc')
plt.show()

**准确率的方差分析**



**准确率的t检验来检查交互模式(简单效应)**

In [35]:
for con in ['pre','unp']:
    df = opt_all[['acc','group','condition']]
    df = df[df['condition']==con][['acc','group']]
    #print(df)
    column_list = [x for x in df.columns if x != 'group']
    # create an empty dictionary
    t_test_results = {}
    # loop over column_list and execute code explained above
    for column in column_list:
        group1 = df.where(df.group== 'High').dropna()[column]
        group2 = df.where(df.group== 'Low').dropna()[column]
        print (group1.count(),group2.count())
        # add the output to the dictionary 
        #print(group1,group2)
        t_test_results[column] = ttest_ind(group1,group2)
    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    print (con,'\n',results_df)

In [36]:
df_bygroup_rt = opt_all[['rt','group','condition']].groupby(['group','condition']).mean()
y_err_rt = opt_all[['rt','group','condition']].groupby(['group','condition']).sem()
df_bygroup_rt = df_bygroup_rt.T
y_err_rt = y_err_rt.T

df_bygroup_rt.plot(kind = 'bar',yerr = y_err_rt,width=0.8,style=['r--','r','b--','b'])#yerr = y_err_v,
plt.legend(loc='upper center', bbox_to_anchor=(1.2,1),ncol=1,fancybox=True,shadow=True)
plt.ylabel('acc')
plt.show()

In [37]:
for con in ['pre','unp']:
    df = opt_all[['rt','group','condition']]
    df = df[df['condition']==con][['rt','group']]
    #print(df)
    column_list = [x for x in df.columns if x != 'group']
    # create an empty dictionary
    t_test_results = {}
    # loop over column_list and execute code explained above
    for column in column_list:
        group1 = df.where(df.group== 'High').dropna()[column]
        group2 = df.where(df.group== 'Low').dropna()[column]
        print (group1.count(),group2.count())
        # add the output to the dictionary 
        #print(group1,group2)
        t_test_results[column] = ttest_ind(group1,group2)
    results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
    results_df.columns = ['statistic','pvalue']
    print (con, '\n',results_df)

In [38]:
for bd_val  in ['acc','rt']:
    df = opt_all[[bd_val,'group','condition']]
    df = df.drop_duplicates()
    color_list = ['rgba(255, 65, 54, 5)', 'rgba(255, 115, 54, 1)',  
                  'rgba(44, 160, 101, 5)', 'rgba(44, 210, 101, 1)']
    scale_list = ['unp','pre','unp','pre']
    group_list = ['High']*2+['Low']*2
    df_gls = pd.DataFrame({'gls':group_list,
                           'sls':scale_list,
                           'cls':color_list})
    #print(df_gls)
#     for group in ['High','Low']:
#         for clms in ['unp','pre']:
#             print(group,clms)
            #print(df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0])

    data = [
                {
                    'y': df[(df['group']==group)&(df['condition']==clms)][bd_val],
                    'type': 'box', 
                    'boxpoints':'all',
                    #'fillcolor':
                    'marker': dict(color = df_gls[(df_gls['gls']==group)&(df_gls['sls']==clms)]['cls'].values[0]),
                    'boxmean':'sd',
                    'name': clms + '_' + group
                }   
                    for group in ['High','Low']
                for clms in ['unp','pre']
            ]
    #iplot(Figure(data=Data(data),layout = Layout(width=500,height=500,title=bd_val,showlegend=True)))


<font face="Arial" size=5 color=#DC143C>使用 dPrime function 来计算信号检测论参数, d,beta,c,Ad. </font> 

In [39]:
from scipy.stats import norm
from math import exp,sqrt
Z = norm.ppf
 
def dPrime(hits, misses, fas, crs):
    # Floors an ceilings are replaced by half hits and half FA's
    halfHit = 0.5/(hits+misses)
    halfFa = 0.5/(fas+crs)
 
    # Calculate hitrate and avoid d' infinity
    hitRate = hits/(hits+misses)
    if hitRate == 1: hitRate = 1-halfHit
    if hitRate == 0: hitRate = halfHit
 
    # Calculate false alarm rate and avoid d' infinity
    faRate = fas/(fas+crs)
    if faRate == 1: faRate = 1-halfFa
    if faRate == 0: faRate = halfFa
 
    # Return d', beta, c and Ad'
    out = {}
    out['d'] = Z(hitRate) - Z(faRate)
    out['beta'] = exp((Z(faRate)**2 - Z(hitRate)**2)/2)
    out['c'] = -(Z(hitRate) + Z(faRate))/2
    out['Ad'] = norm.cdf(out['d']/sqrt(2))
    return out
dPrime(0.8,0.2,0.7,0.3)

In [40]:
#opt_all = opt_all.set_index('Subject')
opt_all['fa'] = opt_all['fa'].apply(lambda x:x/60)
acc_2cond = opt_all[['group','condition','fa','acc']].groupby(['Subject'])['acc'].mean()
opt_all.reset_index(inplace=True) 
opt_all =opt_all.set_index(['Subject'])
opt_all = opt_all.join(acc_2cond,rsuffix='_2cond')
opt_all.reset_index(inplace=True) 
opt_all =opt_all.set_index(['Subject'])
opt_all.reset_index(inplace=True) 
df_dPrime = [dPrime(opt_all['acc_2cond'][idx],1-opt_all['acc_2cond'][idx],opt_all['fa'][idx],1-opt_all['acc_2cond'][idx]) for idx in opt_all.index]
opt_all = opt_all.join(pd.DataFrame(df_dPrime))
opt_all = opt_all.set_index('Subject')

In [41]:
#opt_all[['group','condition','d','c','Ad','beta','fa']].to_csv('tmp.csv')
opt_all.reset_index(inplace=True)
#opt_all = opt_all[opt_all['fa']<=40/60]

In [42]:
# themes =  cf.getThemes()
# print(themes)
# cf.set_config_file(theme=themes[6])
#print(opt_all)
bd_str = 'fa'
#opt_all.to_csv('tmp1.csv')
sublist_na = opt_all['Subject'][opt_all.loc[:,bd_str].isna()].values
df_se_pra_nona = opt_all[~opt_all['Subject'].isin(sublist_na)]
df_tmp = df_se_pra_nona[(df_se_pra_nona['condition']=='unp')][['Subject','group','d','c','Ad','beta','fa']]
df_tmp = df_tmp.drop_duplicates()
df_tmp.to_csv('tmp.csv')
data = [
            {
                'y': df_tmp[(df_tmp['group']==group)][bd_str],
                'type': 'box', 
                'boxpoints':'all',
                'name': bd_str + '_' + str(group)
            } 
                for bd_str in  ['d','c','Ad','beta']
            for group in ['High','Low']
        ]
#iplot(Figure(data=Data(data)))

**行为测量和量表分数间的相关**

In [48]:
#%debug
#bd_str = 'LSHS_AVH'

#opt_all.to_csv('tmp1.csv')
sublist_na = opt_all['Subject'][(opt_all.loc[:,'BDI_all'].isna())|(opt_all.loc[:,'LSHS_AVH'].isna())].values
df_se_pra_nona = opt_all[~opt_all['Subject'].isin(sublist_na)]
df_tmp = df_se_pra_nona[(df_se_pra_nona['condition']=='unp')][['Subject','LSHS_total','LSHS_AVH',  
                                                               'HPSV_all','RAHQ_all',
                                                               'TAI_all','BDI_all','d',  
                                                               'Ad','c','beta','fa']]  
                                                               
df = df_tmp.drop_duplicates()
df.to_csv('tmp.csv')
from scipy.stats import pearsonr
#print(pearsonr(df['LSHS_total'],df['d']))
r_p = [pearsonr(df[cls1],df[cls2]) for cls1 in df.columns for cls2 in df.columns]
r, p = zip(*r_p)
bd_names = ['Subject','LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all',  
            'TAI_all','BDI_all','d','Ad','c','beta','fa']
r = pd.DataFrame(np.array(r).reshape((12,12)),columns=bd_names,index = bd_names)
p = pd.DataFrame(np.array(p).reshape((12,12)),columns=bd_names,index = bd_names)
print(p[['LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all']]) # delete QPE_severity which is empty
#print(p)

In [49]:
import seaborn as sns; sns.set(color_codes=True)
# Set up the matplotlib figure
f, axes = plt.subplots(2, 4, figsize=(15, 15))
sns.despine(left=True)
cnt1 = 0
for clms1 in ['LSHS_AVH','RAHQ_all']:
    cnt2 = 0
    for clms2 in ['d','Ad','c','beta']:#,'fa'
        b= sns.regplot(x=clms1, y=clms2, data=df,ci=68,ax=axes[cnt1, cnt2])
        #sns.set_xlabel("X Label",fontsize=30)
        b.set_xlabel(clms2,fontsize=18*2)
        b.set_ylabel(clms1,fontsize=18*2)
        cnt2 +=1
    cnt1 += 1 
#plt.setp(axes, yticks=[])
plt.tight_layout()

In [50]:
# get a list of all columns in the dataframe without the Group column
df = opt_all[(opt_all['condition']=='unp')][['group','d','c','Ad','beta','fa']]
df.to_csv('tmp.csv')
column_list = [x for x in df.columns if x != 'group']
# create an empty dictionary
t_test_results = {}
# loop over column_list and execute code explained above
for column in column_list:
    group1 = df.where(df.group== 'High').dropna()[column]
    group2 = df.where(df.group== 'Low').dropna()[column]
    # add the output to the dictionary 
    t_test_results[column] = ttest_ind(group1,group2)
results_df = pd.DataFrame.from_dict(t_test_results,orient='Index')
results_df.columns = ['statistic','pvalue']
results_df

In [51]:
opt_all_tmp = opt_all[(opt_all['condition']=='unp')][['Subject','group','d','c','Ad','beta','fa',
                                        'LSHS_total','LSHS_AVH','HPSV_all','RAHQ_all',
                                         'TAI_all','BDI_all',]]
# %store opt_all_tmp
# del opt_all_tmp
# delete QPE_severity which is empty